In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

In [3]:
## 365 변환
def days_to_plus(x):
    return (x*-1)/365

## 업무 시작일 변환
def plus(x):
    if x<0:
        return x*(-1)
    else:
        return 0

## 나이 양수로 변환
trn['birth_plus'] = trn.DAYS_BIRTH.map(days_to_plus)
tst['birth_plus'] = tst.DAYS_BIRTH.map(days_to_plus)
    
## 업무시작일 양수로 변환
trn['employed_plus'] = trn.DAYS_EMPLOYED.map(days_to_plus)
tst['employed_plus'] = tst.DAYS_EMPLOYED.map(days_to_plus)

In [4]:
index_col = 'index'
target_col = 'credit'

cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
float_cols = [x for x in trn.columns.drop('credit') if trn[x].dtype == 'float64']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

21 8 13


In [5]:
from sklearn.preprocessing import LabelEncoder

lbe = LabelEncoder()
for i in cat_cols:
    trn[i] = lbe.fit_transform(trn[i].astype(str))
    tst[i] = lbe.transform(tst[i].astype(str))
    
for i in float_cols:
    trn[i] = trn[i].astype('int')
    tst[i] = tst[i].astype('int')

In [6]:
## 인덱스 설정
trn.set_index('index', inplace = True)
tst.set_index('index', inplace = True)

In [7]:
trn.credit = trn.credit.astype('category')

In [8]:
trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()

In [9]:
## 연봉 제곱 변수 제거
trn6['income_total_2'] = trn6.income_total**2
tst['income_total_2'] = tst.income_total**2

## DAYS_BIRTH 제곱 변수 추가
trn6['DAYS_BIRTH**2'] = trn6.DAYS_BIRTH**2
tst['DAYS_BIRTH**2'] = tst.DAYS_BIRTH**2

## income * days_birth
trn6['income_age']=trn6.income_total*trn6.DAYS_BIRTH
tst['income_age']=tst.income_total*tst.DAYS_BIRTH

## income * DAYS_EMPLOYED
trn6['income_emp']=trn6.income_total*trn6.DAYS_EMPLOYED
tst['income_emp']=tst.income_total*tst.DAYS_EMPLOYED

In [10]:
print(trn6.shape)

(26457, 23)


In [11]:
trn6.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type',
       'begin_month', 'credit', 'birth_plus', 'income_total_2',
       'DAYS_BIRTH**2', 'income_age', 'income_emp'],
      dtype='object')

In [12]:
## 조합 8차시도 : 

# trn6.income_total = np.log1p(trn6.income_total)
trn6['DAYS_BIRTH**2'] = np.log1p(trn6['DAYS_BIRTH**2'])
trn6.income_total_2 = trn6.income_total_2**2
trn6.income_emp = trn6.income_emp**2
trn6.income_age = trn6.income_age**3


# tst.income_total = np.log1p(tst.income_total)
tst['DAYS_BIRTH**2'] = np.log1p(tst['DAYS_BIRTH**2'])
tst.income_total_2 = tst.income_total_2**2
tst.income_emp = tst.income_emp**2
tst.income_age = tst.income_age**3

In [13]:
trn6.income_total = trn6.income_total.astype(int)
trn6['DAYS_BIRTH**2'] = trn6['DAYS_BIRTH**2'].astype(int)

tst.income_total = tst.income_total.astype(int)
tst['DAYS_BIRTH**2'] = tst['DAYS_BIRTH**2'].astype(int)

# Grid

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [42]:
# 그리드
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='accuracy',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=10, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [43]:
ftr=trn6.drop("credit",axis=1)
target=trn6['credit']

# 데이터 분할
X_train,X_val,y_train,y_val=train_test_split(ftr,target,test_size=0.2,random_state=40)

# 모델 객체 생성
model = ExtraTreesClassifier()

paramgrid = {'n_estimators' : [7000],
            'min_samples_split' : [8,9,10,11],
             'max_depth' : [27,28,29,30],
             'max_features' : [3,4,5],
             'random_state' : [2021]
            }

model, pred = algorithm_pipeline(X_train, X_val, y_train, y_val, model, 
                                 paramgrid, cv=5, scoring_fit='accuracy')

print(model.best_score_)
print(model.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0.7106543822348217
{'max_depth': 30, 'max_features': 5, 'min_samples_split': 9, 'n_estimators': 7000, 'random_state': 2021}


In [14]:
## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 10

print(ftr.shape)
print(tst_ar.shape)

cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 42)

ext_p_val = np.zeros((ftr.shape[0], n_class))
ext_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    ext_clf = ExtraTreesClassifier(
                                   max_depth = 30, 
                                   max_features = 4, 
                                   min_samples_split = 9, 
                                   n_estimators = 7000, 
                                   random_state = 2021,
                                  n_jobs = 4)

    ext_clf.fit(ftr[i_trn], target[i_trn])
    ext_p_val[i_val, :] = ext_clf.predict_proba(ftr[i_val])
    ext_p_tst += ext_clf.predict_proba(tst_ar) / n_fold


print(f'{log_loss(target, ext_p_val)}')
print(confusion_matrix(target, np.argmax(ext_p_val, axis=1)))

(26457, 22)
(10000, 22)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10
0.7133101117901619
[[  662   441  2119]
 [  150  2618  3499]
 [  315   887 15766]]


In [15]:
ext_p_val

array([[0.12247438, 0.24397274, 0.63355288],
       [0.69149705, 0.05364093, 0.25486202],
       [0.03620578, 0.31140361, 0.65239061],
       ...,
       [0.12294983, 0.25721616, 0.61983401],
       [0.05752245, 0.23841071, 0.70406684],
       [0.0971995 , 0.23287728, 0.66992321]])

In [27]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'extra'
feature_name = '0.711'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, ext_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, ext_p_tst, fmt='%.6f', delimiter=',')

In [163]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub
sub[sub.columns] = rf_p_tst

In [164]:
sub.to_csv('data/데이터_재표현(전부).csv')

1. trn6.income_age**(3) 으로 했을 때가 0.6866962841415036
2. trn6.income_age**13 으로 했을 때가 0.6873255407581582
*정규분포 형태와 아닌 형태로 대칭인 것만 같고 조금 다름*
=> 직사각형이 더 좋음
3. trn6.income_age 변환 안 함 : 0.6884951810307259
4. tst.income_emp 변환 안 함 :0.6874159266024837
5. tst.income_emp 5제곱으로 직사각형으로 만듦 : 0.6865284163441506
6. trn6.income_total_2**3 으로 제곱을 3으로 변환 : 0.6861786811010409 -> 2제곱이 더 좋음

In [ ]:
## 스태킹에 사용한 extra_2021 : 0.713
ext_clf = ExtraTreesClassifier(
                                   max_depth = 30, 
                                   max_features = 4, 
                                   min_samples_split = 9, 
                                   n_estimators = 7000, 
                                   random_state = 2021,
                                  n_jobs = 4)

In [ ]:
## 스태킹에 사용한 extra_2021_2 : 0.0.711483310787575
ext_clf = ExtraTreesClassifier(
                                   max_depth = 30, 
                                   max_features = 5, 
                                   min_samples_split = 9, 
                                   n_estimators = 7000, 
                                   random_state = 2021,
                                  n_jobs = 4)